In [55]:
import warnings 
warnings.filterwarnings("ignore")

In [56]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Загрузка данных
df = pd.read_csv('2023-11-07_Opros_pervokursnikov_3.csv')  

# Преобразование категориальных признаков в числовой формат
label_encoder = LabelEncoder()
df['Ответ_вопроса_1'] = label_encoder.fit_transform(df['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?'])
df['Ответ_вопроса_2'] = label_encoder.fit_transform(df['В какой тематике вы бы хотели участвовать в научной конференции?'])

# Замена пропущенных значений на средние значения только для числовых столбцов
df.fillna(df.mean(numeric_only=True), inplace=True)

# Присвоение весов ответам на первый вопрос
weights_question_1 = {
    0: 1,  # "Я активно ищу возможности для участия"
    1: 3,  # "Мне интересно, но я ещё не участвовал"
    2: 2,  # "Мне это не интересно"
    3: 4   # "Я не знаю, что такое научные конференции."
}

# Присвоение весов ответам на второй вопрос
weights_question_2 = {
    0: 1, #'Технологии и инновации'
    1: 2, #'Гуманитарные науки'
    2: 2, #'Естественные науки'
    3: 1, #'Социальные науки'
    4: 1 #'Любая тема интересует'
}

# Применение весов к ответам на оба вопроса
df['Важность_вопроса_1'] = df['Ответ_вопроса_1'].map(weights_question_1)
df['Важность_вопроса_2'] = df['Ответ_вопроса_2'].map(weights_question_2)

# Дополнительная логика для формирования кластеров
additional_conditions_target_cluster = (
    (df['Ответ_вопроса_1'].isin([1, 3, 4])) &
    (df['Ответ_вопроса_2'].isin([0, 4]))
)

# Установка важности для ответов, соответствующих дополнительным условиям
df.loc[additional_conditions_target_cluster, 'Важность_вопроса_1'] = 1  # Меняем важность вопроса_1 для соответствующих условий
df.loc[additional_conditions_target_cluster, 'Важность_вопроса_2'] = 1  # Меняем важность вопроса_2 для соответствующих условий

# Кластеризация с использованием Agglomerative Clustering
n_clusters = 5  # Задаем количество кластеров, включая целевой
agg_cluster = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward', affinity='euclidean')
labels_agg = agg_cluster.fit_predict(df[['Ответ_вопроса_1', 'Важность_вопроса_1', 'Важность_вопроса_2']])

# Добавление столбца 'Cluster' с метками кластеров в исходный датасет
df['Cluster'] = labels_agg

# Вывод результатов
df.to_csv('result_df.csv', index=False)


In [57]:
test_question = df[['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?',
                    'В какой тематике вы бы хотели участвовать в научной конференции?', 'Cluster']]
test_question.to_csv('test_question.csv', index=False)

cluster_counts = test_question['Cluster'].value_counts()


print("Standard Clusters:")
print(cluster_counts)



Standard Clusters:
Cluster
2    57
0    45
4    29
1    10
3     9
Name: count, dtype: int64


In [58]:
cluster_distribution = test_question.groupby(['Какое из утверждений наиболее близко описывает ваше отношение к участию в научных конференциях?',
                                            'В какой тематике вы бы хотели участвовать в научной конференции?',
                                            'Cluster', ]).size().reset_index(name='Counts')

total_responses = cluster_distribution['Counts'].sum()
cluster_distribution['Percentage'] = (cluster_distribution['Counts'] / total_responses) * 100

cluster_distribution.to_csv('test2_distribution.csv', index=False)